https://www.youtube.com/watch?v=PCgrgHgy26c&t=128s

In [3]:
  !nvidia-smi

Sun May 28 12:11:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:26:00.0  On |                  N/A |
| 31%   45C    P5    N/A /  75W |    447MiB /  4032MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# First time setup

# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [6]:
# works with colab and not working in jupyter notebook
#!nvcc --version

In [7]:
import torch
torch.__version__

'2.0.1'

In [8]:
#!pip install lightning

In [9]:
import lightning
lightning.__version__

'2.0.2'

## Load data
---

In [ ]:
# Location of the data



## Import libraries
---

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from multiprocessing import cpu_count
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
#from pytorch_lightning.metrics.functional import accuracy
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:

%matplotlib inline
%config InlineBackend.figure_format='retina'

#sns.set(style='whitegrid',palette='mutted',font_scale=1.2)
#HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF7006D", "#ADFF02", "#8F00FF"]
#sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize']=8,5


In [ ]:
pl.seed_everything(42)

In [ ]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
y_train.surface.value_counts().plot(kind="bar")
plt.xticks(rotation=45)
plt.show()

# Preprocessing


In [ ]:
# labling the y_lable

label_encoder = LabelEncoder()
encoded_lables = label_encoder.fit_transform(y_train.surface)

In [ ]:
encoded_lables[:20]

In [ ]:
label_encoder.classes_

In [ ]:
y_train.head()

In [ ]:
y_train["lables"] = encoded_lables

In [ ]:
y_train.head()

In [ ]:
X_train[:130]

In [ ]:
# get list of nessesary feature names in to one array

FEATURE_COLUMNS = X_train.columns.tolist()[3:]
FEATURE_COLUMNS

In [ ]:
X_train.series_id.value_counts()

In [ ]:
(X_train.series_id.value_counts() == 128).sum()

In [ ]:
len(y_train)

In [ ]:
sequences = []

for series_id, group in X_train.groupby("series_id"):
  sequence_features = group[FEATURE_COLUMNS]
  label = y_train[y_train.series_id == series_id].iloc[0].lables
  
  sequences.append((sequence_features,label))

In [ ]:
sequences[4]

In [ ]:
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2)

In [ ]:
len(train_sequences), len(test_sequences)

## Dataset